In [52]:
import pandas as pd
data = pd.read_csv('datasets/retail.csv')
data.head()

,index,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue,Column1
0,0,2/19/2016,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,1.0,80.00,109.000000,80.0,109.0,NaN
1,1,2/20/2016,2016.0,February,29.0,F,United States,Washington,Clothing,Gloves,2.0,24.50,28.500000,49.0,57.0,NaN
2,2,2/27/2016,2016.0,February,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,3.67,5.000000,11.0,15.0,NaN
3,3,3/12/2016,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,2.0,87.50,116.500000,175.0,233.0,NaN
4,4,3/12/2016,2016.0,March,29.0,F,United States,Washington,Accessories,Tires and Tubes,3.0,35.00,41.666667,105.0,125.0,NaN


In [53]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelEncoder

# sequential API allows us to create models layer-by-layer in most of cases
from tensorflow.keras import Sequential
# Dense is the only actual network layer in that model
# 
from tensorflow.keras.layers import Dense

In [54]:
# Drop unnecessary columns
data = data.drop(columns=['index', 'Column1'])

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Fill or drop NaN values
data= data.dropna()

# Ensuring correct data types
data['Customer Age'] = data['Customer Age'].astype(int)
data['Quantity'] = data['Quantity'].astype(int)

n = 1
data.drop(data.tail(n).index,inplace=True) # drop last n rows

# Remove duplicates
data = data.drop_duplicates()

data.head()

,Date,Year,Month,Customer Age,Customer Gender,Country,State,Product Category,Sub Category,Quantity,Unit Cost,Unit Price,Cost,Revenue
0,2016-02-19,2016.0,February,29,F,United States,Washington,Accessories,Tires and Tubes,1,80.00,109.000000,80.0,109.0
1,2016-02-20,2016.0,February,29,F,United States,Washington,Clothing,Gloves,2,24.50,28.500000,49.0,57.0
2,2016-02-27,2016.0,February,29,F,United States,Washington,Accessories,Tires and Tubes,3,3.67,5.000000,11.0,15.0
3,2016-03-12,2016.0,March,29,F,United States,Washington,Accessories,Tires and Tubes,2,87.50,116.500000,175.0,233.0
4,2016-03-12,2016.0,March,29,F,United States,Washington,Accessories,Tires and Tubes,3,35.00,41.666667,105.0,125.0


In [57]:
X = data[['Quantity', 'Unit Cost', 'Unit Price', 'Cost', 'Revenue']].apply(pd.to_numeric)

# encoding the target variable
le = LabelEncoder()
y = le.fit_transform(data['Product Category'])

In [58]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
# Standardize the feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [60]:
# Build the neural network model with sigmoid activation in hidden layers
model = Sequential()
model.add(tf.keras.Input(shape=(5,)))   
model.add(Dense(3, input_dim=2, activation='sigmoid'))  # Sigmoid activation in the hidden layer
model.add(Dense(1, activation='sigmoid')) #output

/Users/semihozturk/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [62]:
# Compile the model, weight optimization
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [63]:
print(X_train.shape)  # Should be (num_samples, input_features)
print(X_test.shape)   # Should be (num_samples, input_features)
model.summary()

(27891, 5)
(6973, 5)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 3)              │            18 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22 (88.00 B)

 Trainable params: 22 (88.00 B)

 Non-trainable params: 0 (0.00 B)

In [64]:
# Label Shapes
print(y_train.shape)
print(y_test.shape)

(27891,)
(6973,)


In [65]:
model.fit(X_train, y_train, epochs=10, batch_size=5, validation_data=(X_test, y_test))

Epoch 1/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 3s 371us/step - accuracy: 0.4595 - loss: 0.6200 - val_accuracy: 0.6945 - val_loss: 0.5198
Epoch 2/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 2s 382us/step - accuracy: 0.6959 - loss: 0.4950 - val_accuracy: 0.7030 - val_loss: 0.4917
Epoch 3/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 2s 359us/step - accuracy: 0.7013 - loss: 0.4648 - val_accuracy: 0.7102 - val_loss: 0.4660
Epoch 4/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 2s 358us/step - accuracy: 0.7087 - loss: 0.4378 - val_accuracy: 0.7031 - val_loss: 0.4471
Epoch 5/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 2s 351us/step - accuracy: 0.7043 - loss: 0.4138 - val_accuracy: 0.7056 - val_loss: 0.4259
Epoch 6/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 2s 351us/step - accuracy: 0.7093 - loss: 0.3855 - val_accuracy: 0.7064 - val_loss: 0.4053
Epoch 7/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 2s 355us/step - accuracy: 0.7032 - loss: 0.3757 - val_accuracy: 0.7076 - val_loss: 0.3783
Epoch 8/10
5579/5579 ━━━━━━━━━━━━━━━━━━━━ 2s 349us/step - accuracy: 0.7165 -

In [66]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 255us/step - accuracy: 0.7287 - loss: 0.2181
Test Loss: 0.2751, Test Accuracy: 0.7235


In [67]:
import numpy as np
# Create a NumPy array with new data for prediction
# example data for: quantity, unit cost, unit price, cost, revenue
new_data = np.array([[10, 15.5, 20.0, 155.0, 200.0]])

# Standardize the new data using the same scaler used during training
new_data_scaled = scaler.transform(new_data)

predictions = model.predict(new_data_scaled)

# get the predicted class
predicted_class = np.argmax(predictions, axis=1)[0]

# decode the predicted class using the LabelEncoder if applicable
decoded_class = le.inverse_transform([predicted_class])[0]

print("Predicted Probabilities:", predictions)
print("Predicted Class:", predicted_class)
print("Decoded Class:", decoded_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted Probabilities: [[0.9850221]]
Predicted Class: 0
Decoded Class: Accessories


/Users/semihozturk/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [68]:
# kNN
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor # this is used for target variable as continuous
from sklearn.neighbors import KNeighborsClassifier # this is used for target variable as categorical
from sklearn.metrics import mean_squared_error

In [69]:
# Target variable is 'Product Category'
X = data[['Quantity', 'Unit Cost', 'Unit Price', 'Cost', 'Revenue']]
y = data['Product Category']

# train/testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#knn_model = KNeighborsRegressor(n_neighbors=4)
#knn_model.fit(X_train, y_train)
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)

In [77]:
y_pred = knn_model.predict(X_test)

In [81]:
# use this for regression,  KNeighborsRegressor
#mse = mean_squared_error(y_test, y_pred) 
#print(f'Mean Squared Error: {mse}')

In [79]:
# Print some actual vs predicted values
print("\nActual vs Predicted Product Category:")
actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(actual_vs_predicted.head(15))


Actual vs Predicted Product Category:
            Actual    Predicted
34433  Accessories  Accessories
29667     Clothing  Accessories
9947   Accessories  Accessories
396    Accessories  Accessories
28004     Clothing  Accessories
2009   Accessories  Accessories
26004  Accessories  Accessories
11683        Bikes  Accessories
15484  Accessories  Accessories
26808  Accessories  Accessories
21127  Accessories  Accessories
16182        Bikes        Bikes
25788  Accessories  Accessories
5562   Accessories  Accessories
7525         Bikes        Bikes
